In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def timeseries_to_network(timeseries, threshold=False, percentile=None):
    corr = timeseries.corr(method="pearson", numeric_only=True)
    correlation_matrix = corr.to_numpy()
    if np.isnan(np.sum(correlation_matrix)):
        return None
    
    if threshold:
        flattened = correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)]
        t = np.percentile(flattened, percentile)
        brain_network = np.zeros(shape=correlation_matrix.shape)
        brain_network[np.where(correlation_matrix > t)] = 1
        return brain_network
    else:
        return correlation_matrix


In [ ]:
src_dir = "../data/ABIDE/timeseries_filt_global/"
base_dest_dir = "../data/generated_filt_global/"

In [4]:
category_dir = base_dest_dir + "pearson_corr_raw/"
if not os.path.exists(category_dir):
    os.makedirs(category_dir)

for dirpath, dirnames, filenames in os.walk(src_dir):

    destination = dirpath.replace(src_dir, category_dir)
    if not os.path.exists(destination):
        os.mkdir(destination)

    for filename in filenames:
        df = pd.read_csv(dirpath + "/" + filename, delim_whitespace=True)
        brain_network = timeseries_to_network(timeseries=df, threshold=False)
        if brain_network is not None:
            np.savetxt(destination + "/" + filename, brain_network, fmt="%f")

In [4]:
for threshold_percentile in [70, 75, 85, 90]:
    
    category_dir = base_dest_dir + f"pearson_corr_thresh_{threshold_percentile}/"
    if not os.path.exists(category_dir):
        os.makedirs(category_dir)

    for dirpath, dirnames, filenames in os.walk(src_dir):

        destination = dirpath.replace(src_dir, category_dir)
        if not os.path.exists(destination):
            os.mkdir(destination)

        for filename in filenames:
            df = pd.read_csv(dirpath + "/" + filename, delim_whitespace=True)
            brain_network = timeseries_to_network(timeseries=df, threshold=True, percentile=threshold_percentile)
            if brain_network is not None:
                np.savetxt(destination + "/" + filename, brain_network, fmt="%i")